In [ ]:
import os, sys, subprocess, pickle, tifffile, copy
import numpy as np
import scipy
from datetime import datetime
import matplotlib.pyplot as plt
# sys.path.insert(0, '/asap3/petra3/gpfs/common/p21.2/scripts/')
sys.path.insert(0, '/home/sjoehann/')
import pyTSXRD
from pyTSXRD.angles_and_ranges import merge_overlaps
from pyTSXRD.DataAnalysis import merge_DATA_list, plot_sinogram, compute_sinogram
import experiment_settings

from scipy.optimize import fsolve
from scipy.spatial.distance import cdist
from orix import plot, sampling
from orix.crystal_map import Phase
from orix.quaternion import Orientation, symmetry
from orix.vector import Vector3d
from scipy.ndimage.interpolation import rotate
from hexrd import imageseries
from hexrd.imageseries.omega import OmegaWedges
from numpy import float32
import fabio

single_separator = "--------------------------------------------------------------"
double_separator = "=============================================================="

### SETTING GLOBAL VARIABLES - DIRECTORY ROOT AND MATERIAL:
# path_gen = '/asap3/petra3/gpfs/p21.2/2022/data/11013744/'
path_gen = '/asap3/petra3/gpfs/p21.2/2023/data/11016686/' #raw/polyAu04/038_y1_0.0000/Varex_4/' 022_y1_-1.6000.fio
materials_table = {'Au'  : {'name': 'Au'  , 'spacegroup': 225, 'symmetry':'F', 'unitcell': [4.0782, 4.0782,  4.0782, 90., 90.,  90.]},
                   'CeO2': {'name': 'CeO2', 'spacegroup': 225, 'symmetry':'F', 'unitcell': [5.4115, 5.4115,  5.4115, 90., 90.,  90.]},
                   'Cu'  : {'name': 'Cu'  , 'spacegroup': 225, 'symmetry':'F', 'unitcell': [3.5942, 3.5942,  3.5942, 90., 90.,  90.]},
                   'LaB6': {'name': 'LaB6', 'spacegroup': 221, 'symmetry':'P', 'unitcell': [4.1568, 4.1568,  4.1568, 90., 90.,  90.]},
                   'MgCa': {'name': 'MgCa', 'spacegroup': 194, 'symmetry':'P', 'unitcell': [3.1980, 3.1980,  5.1900, 90., 90., 120.]},
                   'Nb'  : {'name': 'Nb'  , 'spacegroup': 229, 'symmetry':'I', 'unitcell': [3.3042, 3.3042,  3.3042, 90., 90.,  90.]},
                   'Ni'  : {'name': 'Ni'  , 'spacegroup': 225, 'symmetry':'F', 'unitcell': [3.5100, 3.5100,  3.5100, 90., 90.,  90.]},
                   'Pd'  : {'name': 'Pd'  , 'spacegroup': 225, 'symmetry':'F', 'unitcell': [3.8907, 3.8907,  3.8907, 90., 90.,  90.]},
                   'Ruby': {'name': 'Ruby', 'spacegroup': 167, 'symmetry':'R', 'unitcell': [4.7608, 4.7608, 12.9957, 90., 90., 120.]},
                   'Ti'  : {'name': 'Ti'  , 'spacegroup': 194, 'symmetry':'P', 'unitcell': [2.9505, 2.9505,  4.6826, 90., 90., 120.]}}

material = materials_table['Pd']

### SETTING THE SWEEPSCANS PARAMETERS, AND OTHER OBJECTS:
def set_SweepProcessor(path_gen, i_load, i_slow, i_fast, det_num, material):
    default_xyz = [0,0,0]
    y_points = np.linspace(-3.6, 3.6, 73) # y-motor positions if needed
    meta_key = path_gen + f'raw/polyPd05/{i_fast:03d}_y1_{y_points[i_fast]:.4f}.fio' # metadata of the sweep    
    S = experiment_settings.set_p212_sweep(path_gen, i_slow, i_fast, det_num, default_xyz, meta_key) # this function usually don't need modifications
    if S.sweep['stem'][-1] != '_': S.sweep['stem'] += '_' # correct for possible mismatch between file stems in the sweep command and actual file names
    S.processing['options'] = None # ('flip', 'r270') # detector flips if needed to be applied before those in geometry file
    S.directory = S.directory.replace('/polyPd05/', '/hannasjo/polyPd05/') # if needed to modify the output directory (ie when the default one must not be overwritten)
    S.geometry.load_par(directory = path_gen + 'processed/hannasjo/', par_file = f'Pd_pyFAI_calib.par')
#     S.geometry.load_yml(directory = path_gen + 'processed/Anatoly/', yml_file = f'Au_calib.yml')
    S.geometry.set_attr('material', material)
    S.geometry.spline_file = None # Either None for perfect detector, or path to spline file
    S.geometry.set_attr('wavelength' , 12.39842/38)
    return S

#TOLERANCES USED IN MORE THAN ONE FUNCTION
tth_ranges = [[8.0, 16.3]] #limit around where you have your rings of interest 
eta_ranges = [[ -90, -5], [5, 90]]
omega_ranges = [-180,  180]
ds_ranges = []

tth_gap = 0.5 # tolerances for ranges of evaluating gvectors
ds_gap = 0.1 # tolerances for ranges of evaluating gvectors
eta_gap = 1 # tolerances for ranges of evaluating gvectors

dens_omg_tol = 1.0  #tolerance for creating density map 
dens_eta_tol = 1.1
dens_tth_tol = 0.5  

def set_GrainSpotter(path_gen, material,tth_ranges,eta_ranges,omega_ranges):
    GS = experiment_settings.set_grainspotter(path_gen, material, domega=None)
    GS.set_attr('tth_ranges'   , tth_ranges) # 12.7]])
    GS.set_attr('ds_ranges'    , [] ) # [0.5, 1.0]]) # GV.ds_ranges)
    GS.set_attr('eta_ranges'   , merge_overlaps( eta_ranges, margin=0, target=0)  ) # GV.eta_ranges)
    GS.set_attr('omega_ranges' , [ omega_ranges]) # GV.omega_ranges)
    GS.set_attr('cuts'         , [ 12, 0.6,  0.6] )
    GS.set_attr('uncertainties', [0.5, 1.8,  1.5] ) # [sigma_tth sigma_eta sigma_omega] in degrees
    GS.set_attr('nsigmas'      , 1)
    GS.set_attr('eulerstep'    , 6)
    GS.set_attr('Nhkls_in_indexing', None)
    GS.set_attr('random', 10000)
    GS.set_attr('positionfit', True)
    return GS


def set_PolySim(path_gen, grainspotter = None, material = None):
    PS = experiment_settings.set_polyxsim(grainspotter, material)
    PS.set_attr('inp_file', grainspotter.log_file.strip('.log'))
    PS.set_attr('beamflux', 1e12)
    PS.set_attr('beampol_factor', 1)
    PS.set_attr('beampol_direct', 0)
    PS.set_attr('stem'  , grainspotter.log_file.replace('.log', '_sim'))
    PS.set_attr('grains', [])
    PS.set_attr('omega_start', grainspotter.omega_ranges[0][0])
    PS.set_attr('omega_step' , abs(grainspotter.domega))
    PS.set_attr('omega_end'  , grainspotter.omega_ranges[-1][1])
    PS.set_attr('theta_min'  , grainspotter.tth_ranges[0][0]/2)
    PS.set_attr('theta_max'  , grainspotter.tth_ranges[-1][1]/2)
    PS.set_attr('no_grains'  , 1)
    PS.set_attr('gen_U'   , 0)
    PS.set_attr('gen_pos' , [0, 0])
    PS.set_attr('gen_eps' , [1, 0, 0 ,0, 0])
    PS.set_attr('gen_size', [0.0, 0.0, 0.0 ,0.0])
    PS.set_attr('make_image', 0)
    PS.set_attr('output', ['.tif', '.par', '.gve'])
    PS.set_attr('bg' , 0)
    PS.set_attr('psf', 0.7)
    PS.set_attr('peakshape', [1, 4, 0.5])
    return PS


def set_DATA(path_gen, i_load, i_slow, i_fast, detectors, material):
    yml_det_order = [1] # [4,1,2,3]
    DATA = pyTSXRD.DataAnalysis()
    DATA.set_attr('material', material)
    for det_num in detectors:
        S = set_SweepProcessor(path_gen, i_load, i_slow, i_fast, det_num, material)
        DATA.add_to_attr('sweepProcessors', S)
    DATA.set_attr('yml_det_order', yml_det_order)
    DATA.set_attr('directory'    , S.directory)
    DATA.set_attr('name'         , f's{i_slow:03d}_f{i_fast:03d}_'+material['name'])
    DATA.set_attr('position'     , S.position)
    DATA.set_attr('rotation'     , [0,0,0])
    DATA.set_attr('sample_pix_x',np.linspace(-3.5, 3.5, 701)) #sample size and pixels used when creating map
    DATA.set_attr('sample_pix_y',np.linspace(-3.5, 3.5, 701))
    DATA.set_attr('beamsize', 0.1) #beamsize in mm
    if S.log_meta:
        load_values = [ent['load'] for ent in S.log_meta['entries']]
        DATA.set_attr('pressure', sum(load_values) / len(load_values))
    return DATA

def set_paths(path_gen, load_states, slow_translations, fast_translations, detectors, material):
    list_DATApaths = []
    GE_list = []
    for i_load in load_states[:]: # 
        for i_slow in slow_translations[:]:     # (e.g. idty1).
            for i_fast in fast_translations[:]: # (e.g. idtz2).
                DATA = set_DATA(path_gen, i_load, i_slow, i_fast, detectors, material)
                DATA = pickle.load(open(DATA.directory + DATA.name + "_DATA.p","rb") )
                for g in DATA.gvectorEvaluator.gvectors:
                    g['stage_x'], g['stage_y'], g['stage_z'] = DATA.position
                pickle.dump(DATA, open(DATA.directory+DATA.name+"_DATA.p","wb") )
                list_DATApaths.append(DATA.directory+DATA.name+"_DATA.p")
                GE_list.append(DATA.gvectorEvaluator)    
    return list_DATApaths, GE_list

load_states = [0] # Indices of loads to analyze
slow_translations = [0]            # Indices of positions to analyze
fast_translations = list(range(73)) # Indices of positions to analyze
detectors         = [4]

### HERE WE SET TTH-DEPENDENT THRESHOLD
def parab_eq(a, x):
    return a[0]*x*x + a[1]*x + a[2]
def func(a):
    datapoints = [( 5, 800), (11, 600), (15, 300)]  # 800 cts at 5 degrees, 600 at 11, and 300 at 15
    return [parab_eq(a,x) - y for x,y in datapoints]
a = fsolve(func, [1, 1, 1])
def thr(tth):
    return parab_eq(a, tth)

In [ ]:
"""TEST FOR CENTER ROTATION. LOADING RAW DATA. PEAKSERCHING, MERGING, APPLYING INSTRUMENT CONFIGURATION."""
# Test for the middle point to see if parameters are good. Here we print and plot and if we have a good sample we should see at least one grain
slow_mid = len(slow_translations)//2 
fast_mid = len(fast_translations)//2
for i_load in load_states[0:1]: # 
    for i_slow in slow_translations[slow_mid:slow_mid+1]:     # (e.g. idty1).
        for i_fast in fast_translations[fast_mid:fast_mid+1]: # (e.g. idtz2).
            print(double_separator + f'\nLOAD = {i_load}, TRANSLATIONS: slow = {i_slow}, fast = {i_fast}')
            DATA = set_DATA(path_gen, i_load, i_slow, i_fast, detectors, material)  
            DATA.process_images(frames = 'all', save_tifs = False, q0_pos = 'auto', rad_ranges = 'auto', thr = 'auto') #load and process images
            DATA.peaksearch(peaksearch_thrs = 'auto', peakmerge_thrs = 'auto', min_peak_dist = 10) #search for peaks 
            DATA.index(thr = thr) #index peaks 
            DATA.gvectorEvaluator.remove_not_ranges(ds_ranges = [], tth_ranges=tth_ranges, omega_ranges=[omega_ranges], eta_ranges=eta_ranges) #only concider good rings
            ds_tol = DATA.gvectorEvaluator.merged[0].peakIndexer.geometry.ds_from_tth(0.12)
            DATA.evaluateGvectors(ds_tol=ds_tol, tth_gap=tth_gap, ds_gap=ds_gap, eta_gap=eta_gap) # these params only for calculating tth and eta ranges
            DATA.searchGrains(grainSpotter = set_GrainSpotter(path_gen, material,tth_ranges,eta_ranges,omega_ranges)) #find grains 
            DATA.runPolyXSim(polyxsim = set_PolySim(path_gen, DATA.grainSpotter, material),GE_SIM_list=[DATA.gvectorEvaluator.merged[0]],also_plot=True)
            pickle.dump(DATA, open(DATA.directory+DATA.name+"_DATA.p","wb") )        

In [ ]:
"""LOADING RAW DATA. PEAKSERCHING, MERGING, APPLYING INSTRUMENT CONFIGURATION."""
# For all points 
for i_load in load_states[0:1]: # 
    for i_slow in slow_translations[:]:     # (e.g. idty1).
        for i_fast in fast_translations[:]: # (e.g. idtz2).
            print(double_separator + f'\nLOAD = {i_load}, TRANSLATIONS: slow = {i_slow}, fast = {i_fast}')
            DATA = set_DATA(path_gen, i_load, i_slow, i_fast, detectors, material)            
            DATA.process_images(frames = 'all', save_tifs = False, q0_pos = 'auto', rad_ranges = 'auto', thr = 'auto') #load and process images
            DATA.peaksearch(peaksearch_thrs = 'auto', peakmerge_thrs = 'auto', min_peak_dist = 10) #search for peaks 
            DATA.index(thr = thr) #index peaks 
            DATA.gvectorEvaluator.remove_not_ranges(ds_ranges = [], tth_ranges=DATA.grainspotter['tth_ranges'], omega_ranges=DATA.grainspotter['omega_ranges'], eta_ranges=DATA.grainspotter['eta_ranges']) #only concider good rings
            ds_tol = DATA.gvectorEvaluator.merged[0].peakIndexer.geometry.ds_from_tth(0.12)
            DATA.evaluateGvectors(ds_tol=ds_tol, tth_gap=tth_gap, ds_gap=ds_gap, eta_gap=eta_gap) # these params only for calculating tth and eta ranges
            #DATA.searchGrains(grainSpotter = set_GrainSpotter(path_gen, material)) #find grains 
            #DATA.runPolyXSim(polyxsim = set_PolySim(path_gen, DATA.grainSpotter, material),GE_SIM_list=[DATA.gvectorEvaluator.merged[0]])
            pickle.dump(DATA, open(DATA.directory+DATA.name+"_DATA.p","wb") )            

In [ ]:

list_DATApaths, GE_list = set_paths(path_gen, load_states, slow_translations, fast_translations, detectors, material)
ind_mid = int(len(list_DATApaths)/2)
DATA  = copy.deepcopy(pickle.load(open(list_DATApaths[ind_mid], "rb")))
DATA.set_attr('directory', DATA.directory.replace(DATA.directory.split('/')[-2]+'/', '') )
DATA.set_attr('name', 's000_all_filtered')
DATA.set_attr('sample_pix_x',np.linspace(-3.5, 3.5, 701)) #sample size and pixels used when creating map
DATA.set_attr('sample_pix_y',np.linspace(-3.5, 3.5, 701))
DATA.set_attr('beamsize', 0.1) 
DATA.set_attr('plot_range' ,[50,150,250,350,450,550,650])
DATA.set_attr('label_range',[-3,-2,-1,0,1,2,3])
#Merge data and remove peaks outside of ranges
DATA_ALL = merge_DATA_list(DATA, list_DATApaths, spot3d_id_reg = 5000000)
DATA_ALL.print()
DATA_ALL.gvectorEvaluator.calc_histo(omega_pixsize=0.5,eta_pixsize=0.5,ds_eta_omega_file = None,plot=False,save_arrays = False)
#Calc. sinogram 
#sinogram, pos_linspace, omg_linspace, eta_linspace = compute_sinogram('test', list_DATApaths, pos_step =0.1, omg_step = 0.25, eta_step = 0.1,
                                     	#ds_ranges = 'auto', tth_ranges = tth_ranges,omega_ranges = [omega_ranges],eta_ranges = eta_ranges,save_sinogram=False)

In [ ]:
"""SEACHING FOR GRAINS"""
#Runs commands several times, removes used vectors. Can find more grains but slow
g_list = []
list_g_lists = []
gve_list = copy.copy(DATA_ALL.gvectorEvaluator.gvectors)
spot3d_id_to_remove = []

 # tolerances for grouping gvectors
group_y_tol = 0.5
group_tth_tol = 0.5 
group_eta_tol = 0.18 
group_omega_tol = 1.5


for itr in range(16):
    print(double_separator)
    print('Iteration: ',itr)
    print(double_separator)
    GE_SIM_list = []
    for GE in DATA_ALL.gvectorEvaluator.merged:
        GE_SIM_list += GE.merged
    DATA_ALL.set_attr('grains', [])
    DATA_ALL.gvectorEvaluator.set_attr('gvectors', [gve for gve in gve_list if gve['spot3d_id'] not in spot3d_id_to_remove])
    DATA_ALL.gvectorEvaluator.group_gvectors(group_y_tol=group_y_tol, group_tth_tol=group_tth_tol, group_eta_tol=group_eta_tol, group_omega_tol=group_omega_tol)
    DATA_ALL.evaluateGvectors(ds_tol='auto', tth_gap=tth_gap, ds_gap=ds_gap, eta_gap=eta_gap)
    DATA_ALL.searchGrains(grainSpotter = set_GrainSpotter(path_gen, material,tth_ranges,eta_ranges,omega_ranges))
    DATA_ALL.runPolyXSim(polyxsim = set_PolySim(path_gen, DATA_ALL.grainSpotter, material), GE_SIM_list = [DATA_ALL.gvectorEvaluator.merged[ind_mid].merged[0]])
    for g in DATA_ALL.grains:
        g.compute_density_map(GE_SIM_list, DATA_ALL.sample_pix_x, DATA_ALL.sample_pix_y, DATA_ALL.beamsize, 
                              dens_omg_tol, dens_eta_tol, dens_tth_tol, support_thr=12,final_map=False,bigbeam=False)
    for g in DATA_ALL.grains:
        for gvm in g.gvectors:
            for gve in gve_list:
                if gvm['stage_y'] == gve['stage_y']:
                    if gvm['omega'] == gve['omega']:
                        GE, p = DATA_ALL.gvectorEvaluator.trace_peak_by_spot3d_id(gve['spot3d_id'])
                        if p['spot3d_id'] == gvm['spot3d_id']:
                            spot3d_id_to_remove.append(gve['spot3d_id'])
    g_list += [g for g in DATA_ALL.grains if len(g.gvectors) > 0]
    list_g_lists += [DATA_ALL.grains]

DATA_ALL.set_attr('grains', g_list)
n_grains = len(DATA_ALL.grains)
print('Number of grains:', n_grains)
pickle.dump(DATA_ALL, open(DATA_ALL.directory+DATA_ALL.name+"_DATA_ALL.p","wb") )

In [ ]:
"""LOAD DATA IF NEEDED"""
DATA_ALL = set_DATA(path_gen, 0, 0, 0, detectors, material)
DATA_ALL.set_attr('name','s000_all_filtered')
DATA_ALL.set_attr('directory', DATA_ALL.directory.replace(DATA_ALL.directory.split('/')[-2]+'/', '') )
file = open(DATA_ALL.directory + DATA_ALL.name + "_DATA_ALL.p","rb")
DATA_ALL = pickle.load(file)
file.close()

In [ ]:
"""REMOVE DUPLICATES"""
n_grains = len(DATA_ALL.grains)
print('Number of grains:', n_grains )
DATA_ALL.remove_duplicates(ang_tol=.5, pos_tol=0.1) #If set too high it can reove too many grains

In [ ]:
"""CREATE DENSITY MAP FOR ALL GRAINS FOUND IN THE PREVIOUS STEP"""
GE_SIM_list = []
n_grains = len(DATA_ALL.grains)
for GE in DATA_ALL.gvectorEvaluator.merged:
    GE_SIM_list += GE.merged
for i,g in enumerate(DATA_ALL.grains):
    g.compute_density_map(GE_SIM_list, DATA_ALL.sample_pix_x, DATA_ALL.sample_pix_y, DATA_ALL.beamsize, 
                          dens_omg_tol, dens_eta_tol, dens_tth_tol, support_thr=8,sample_rot=0,final_map=True,bigbeam=False)
    print('Density maps %.2f%% done   ' %((i+1)/n_grains*100), end="\r")
    
pickle.dump(DATA_ALL, open(DATA_ALL.directory+DATA_ALL.name+"_DATA_ALL.p","wb") )

In [ ]:
"""LOAD DATA IF NEEDED"""
DATA_ALL = set_DATA(path_gen, 0, 0, 0, detectors, material)
DATA_ALL.set_attr('name','s000_all_filtered')
DATA_ALL.set_attr('directory', DATA_ALL.directory.replace(DATA_ALL.directory.split('/')[-2]+'/', '') )
file = open(DATA_ALL.directory + DATA_ALL.name + "_DATA_ALL_5.p","rb")
DATA_ALL = pickle.load(file)
file.close()

In [ ]:
"""PLOT DENSITY MAPS"""
DATA_ALL.plot_dmap(DATA_ALL.sample_pix_x,DATA_ALL.sample_pix_y,grain_number=28,plot_type = 'simple',also_save=True,with_colorbar=False)

In [ ]:
"""TAKES DENSITY MAP AND RETURNS A MATRIX FOR EACH GRAIN""" 
completeness_th=0.45 #th for masking of the density map 
DATA_ALL.make_grainmatrix(completeness_th=completeness_th,also_plot=False,save_matrix=False)

In [ ]:
"""CORRECT FOR TILT OF SAMPLE"""
mu = -0.09
DATA_ALL.tilt_correction(ang_inc=mu)

In [ ]:
"""Define values we need convert plot from pixel values to mm"""
DATA_ALL.set_attr('plot_range' ,[50,150,250,350,450,550,650])
DATA_ALL.set_attr('label_range',[-3,-2,-1,0,1,2,3])

#REMOVE PARTS OF GRAINS OUTSIDE OF GRAINBOUNDARIES 
radius = 701/2
DATA_ALL.apply_samplemask(radius,also_plot=False,plot_invers=False,save_invers=False)

#TAKE THE MOST COMPLETE GRAIN IN EACH POINT WHERE GRAINS OVERLAPP. OPTION TO PLOT & SAVE
grain_th = 100 #If a grain has a total number of pixels below the th, the grain will be remove
DATA_ALL.map_sample(also_plot=False,grain_th=grain_th)

#PLOT AS AN INVERSE POLEFIGURE MAP
name_file = 'pd_1' #extension to file name of map
DATA_ALL.plot_colors(name_file,plot_type='full',also_save=False,mark_grainnumber=False,mark_millers=False)

pickle.dump(DATA_ALL, open(DATA_ALL.directory+DATA_ALL.name+"_DATA_MAP.p","wb") )

In [ ]:
"""SOME GRAINS MIGHT BE SPLIT INTO TWO OR MORE PARTS, HERE THEY CAN BE COMBINED"""
ang_tol = 1.73 #threshold to still be concidered the same grain
pos_tol = 15 #distance in pixel index of any part of the grain to still be concidered the same grain
DATA_ALL.combine_duplicates(ang_tol, pos_tol)

#PLOT AS AN INVERSE POLEFIGURE MAP
name_file = 'pd_2' #extension to file name of map
DATA_ALL.plot_colors(name_file,plot_type='full',also_save=True,mark_grainnumber=False,mark_millers=False)
pickle.dump(DATA_ALL, open(DATA_ALL.directory+DATA_ALL.name+"_DATA_MAP.p","wb") )